In [4]:
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
import subfunc as sf
from tqdm import tqdm
from collections import defaultdict

In [5]:
# Models fit flag
is_fitted = False

In [6]:
DATA_PATH = 'data/'
FINAL_STAGE = 'final_stage/'
X_train = pd.read_csv(f'{DATA_PATH}{FINAL_STAGE}x_train.csv',sep='\t', index_col='index')
X_test = pd.read_csv(f'{DATA_PATH}{FINAL_STAGE}x_test.csv', sep='\t', index_col='index')
y_train = pd.read_csv(f'{DATA_PATH}{FINAL_STAGE}y_train.csv',sep='\t', index_col='index')
y_test = pd.read_csv(f'{DATA_PATH}{FINAL_STAGE}y_test.csv',sep='\t', index_col='index')

In [11]:
np.random.seed(42)
models = defaultdict()

if not is_fitted:
    best_score = 0
    best_model = None
    for n_estimators in np.arange(250, 350, 50):
        for lr in np.arange(0.001, 0.1, 0.005):
            current_model  = GradientBoostingRegressor(
                n_estimators=n_estimators,
                learning_rate=lr)
            current_model.fit(X_train, y_train.values.ravel())
            current_score = r2_score(y_test.values.ravel(), current_model.predict(X_test))

            if current_score > best_score:
                best_score = current_score
                best_model = current_model
                print(f'{best_score}')

    models['model'] = best_model
    sf.models_save(models)
else:
    models = sf.models_load()

0.11192767412390403
0.30664029540218785
0.35681165104427726
0.37587455469856257
0.3871281699337109
0.389393195072249
0.3916752048092077
0.39221602389589194
0.39376099618425053
0.3963944970581804
0.39769808605590296
0.39996723083220986
0.40117397224148676
0.4038678889200016
0.40655733096570845
